In [3]:
#ElasticNet Method
#Import Libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.linear_model import ElasticNet, ElasticNetCV
from sklearn import datasets


In [4]:
#Import Data and Extract Features and Dependend Variable
train = pd.read_csv("./data/cleaned_standardized_fe.csv")
features = train.drop('SalePrice', axis = 1)
price = train['SalePrice'] 

In [17]:
#Cross Validate Lasso Model to Select Best alpha
alphas = np.linspace(.0001, 1, 100)

elastic_cv = ElasticNetCV(alphas = alphas, cv = 10, max_iter = 10000, normalize = False, l1_ratio = .5,)
elastic_cv.fit(features,price)
print('the best alpha is: %.5f' %(elastic_cv.alpha_))

#Fit Model on complete test set using best alpha and get R^2 score and print coeficients
elastic = ElasticNet()
elastic.set_params(normalize=False)
elastic = ElasticNet(alpha = elastic_cv.alpha_)
elastic.fit(features, price)
print('the R^2 Score is: %.2f' %(elastic.score(features, price)))
pd.Series(elastic.coef_, index=features.columns)

the best alpha is: 0.02030
the R^2 Score is: 0.91


LotFrontage             -0.092810
LotArea                  0.000000
MasVnrArea              -0.000000
HeatingQC               -0.037002
LowQualFinSF            -0.005281
GrLivArea               -0.044218
FullBath                -0.081153
HalfBath                -0.032604
BedroomAbvGr            -0.000000
TotRmsAbvGrd            -0.025367
Functional              -0.043181
GarageCars              -0.091560
MSZoning_Residential    -0.070631
LotShape_Reg             0.000000
LandContour_NotLvl       0.000000
LotConfig_CulDSac       -0.014776
LotConfig_Inside         0.000000
LotConfig_Other          0.004154
LandSlope_Mod           -0.009358
LandSlope_Sev            0.000000
Condition1_Norm         -0.046860
Condition1_Pos          -0.000000
Condition1_RR            0.000000
HouseStyle_1.5Unf        0.000000
HouseStyle_1Story        0.000000
HouseStyle_2.5Fin        0.000000
HouseStyle_2.5Unf       -0.000000
HouseStyle_2Story        0.000000
HouseStyle_SFoyer       -0.000000
HouseStyle_SLv

In [18]:
test = pd.read_csv("./data/cleaned_standardized_TEST.csv")
train_pre_stand = pd.read_csv("./data/train.csv")
submission = pd.read_csv("./data/sample_submission.csv")
sale_price_pred = elastic.predict(test)
submission['SalePrice'] = sale_price_pred
submission.head()

,Id,SalePrice
0,1461,0.791008
1,1462,0.105242
2,1463,-0.326387
3,1464,-0.605935
4,1465,-0.591971


In [19]:
#inverse transform
from sklearn.preprocessing import StandardScaler
#Re-Train Scaler
scaler = StandardScaler()
scaler = scaler.fit(train_pre_stand[['SalePrice']]**-.1)


# inverse transform on transformed price
submission['SalePrice'] = (scaler.inverse_transform(submission['SalePrice'])**-10).round(0)
submission.to_csv("./data/elastic_submission.csv",index = False)

In [20]:
submission['SalePrice'].head()

0    121251.0
1    158613.0
2    188539.0
3    211212.0
4    210011.0
Name: SalePrice, dtype: float64